In [65]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [66]:
jm1 = pd.read_csv(r'/content/jm1_csv.csv')
jm1=jm1.replace('?',0)
print('jm1 shape: ', jm1.shape)
jm1.head()

jm1 shape:  (10885, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1,1,1,1,1,True
2,72.0,7.0,1.0,6.0,198.0,1134.13,0.05,20.31,55.85,23029.10,...,51,10,8,1,17,36,112,86,13,True
3,190.0,3.0,1.0,3.0,600.0,4348.76,0.06,17.06,254.87,74202.67,...,129,29,28,2,17,135,329,271,5,True
4,37.0,4.0,1.0,4.0,126.0,599.12,0.06,17.19,34.86,10297.30,...,28,1,6,0,11,16,76,50,7,True


In [67]:
jm1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10885 entries, 0 to 10884
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   loc                10885 non-null  float64
 1   v(g)               10885 non-null  float64
 2   ev(g)              10885 non-null  float64
 3   iv(g)              10885 non-null  float64
 4   n                  10885 non-null  float64
 5   v                  10885 non-null  float64
 6   l                  10885 non-null  float64
 7   d                  10885 non-null  float64
 8   i                  10885 non-null  float64
 9   e                  10885 non-null  float64
 10  b                  10885 non-null  float64
 11  t                  10885 non-null  float64
 12  lOCode             10885 non-null  int64  
 13  lOComment          10885 non-null  int64  
 14  lOBlank            10885 non-null  int64  
 15  locCodeAndComment  10885 non-null  int64  
 16  uniq_Op            108

In [68]:
target = 'defects'
y = jm1[target]
x=jm1.drop(['defects'], axis=1)
x.shape

(10885, 21)

In [69]:
y.shape

(10885,)

after running " *from mlxtend.feature_selection import SequentialFeatureSelector as SFS* " this command you might get error this is because mlxtend is trying to import joblib from sklearn but it no longer exist there so fix this issue go to the file location (there will a file location link in the error message like this " /usr/local/lib/python3.7/dist-packages/mlxtend/feature_selection/sequential_feature_selector.py <module> " click on this location link)
then it will show you code written in that file then you just have to look for this line " from sklearn.externals.joblib import Parallel, delayed " and edit it to " from joblib import Parallel, delayed " and then save it by pressing ctrl+s. then run the code again. Again it will give error but this time the location link will be different and you have to again edit the file in same way as done priviouslly and save it and run the code again and this will solve the error.

In [70]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.svm import SVC
svc = SVC()


In [71]:
efs = EFS(svc, # can use any any learning method 
          min_features=1, # can set any limit from 1 to less then or equal to max_feature value
          max_features=4, # can set any value less then toatal number of features but greater then or equal to min_features
                          # higer the difference between the min and max value higher the time taken to select features
          scoring='accuracy',
          print_progress=True,
          cv=2
          ).fit(np.array(x[x.columns[0:5]].fillna(0)), y)
                                    # set value of columns like [0:x] where x is equal to value of max_features

Features: 30/30

In [72]:
efs.best_idx_

(3,)

In [73]:
x_reduced = efs.transform(x)
x_reduced.shape

(10885, 1)

In [74]:
y.shape

(10885,)

In [75]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y, test_size=0.3, random_state=10)

In [76]:
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(10885, 21)
(10885,)
-------------------
(7619, 1)
(7619,)
(3266, 1)
(3266,)


In [77]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.7991426821800367
Th confusion matrix of SVM:
  [[2594   16]
 [ 640   16]]


In [78]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[2594   16]
 [ 640   16]]
              precision    recall  f1-score   support

       False       0.80      0.99      0.89      2610
        True       0.50      0.02      0.05       656

    accuracy                           0.80      3266
   macro avg       0.65      0.51      0.47      3266
weighted avg       0.74      0.80      0.72      3266



In [79]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.7976117575015309
The confusion matrix of KNN:
  [[2572   38]
 [ 623   33]]
[[2572   38]
 [ 623   33]]
              precision    recall  f1-score   support

       False       0.81      0.99      0.89      2610
        True       0.46      0.05      0.09       656

    accuracy                           0.80      3266
   macro avg       0.63      0.52      0.49      3266
weighted avg       0.74      0.80      0.73      3266



In [80]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.7930189834660135
[[2552   58]
 [ 618   38]]
              precision    recall  f1-score   support

       False       0.81      0.98      0.88      2610
        True       0.40      0.06      0.10       656

    accuracy                           0.79      3266
   macro avg       0.60      0.52      0.49      3266
weighted avg       0.72      0.79      0.73      3266



In [81]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.813492584328652

In [82]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

       False       0.81      0.97      0.88      2610
        True       0.47      0.10      0.17       656

    accuracy                           0.80      3266
   macro avg       0.64      0.54      0.53      3266
weighted avg       0.74      0.80      0.74      3266

[[2537   73]
 [ 590   66]]
